In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import librosa
import librosa.feature
import glob
import numpy as np
from utils import load_data
from models import simple_cnn

## Data preprocessing

In [7]:
trainX, trainY, validationX, validationY, testX, testY = load_data(split_into_k_equal_parts=True, k=5)

In [8]:
#Reshaping into (batch_size, height, width, channels)
trainX = trainX.reshape((trainX.shape[0], trainX.shape[1], trainX.shape[2], 1))
validationX = validationX.reshape((validationX.shape[0], validationX.shape[1], validationX.shape[2], 1))
testX = testX.reshape((testX.shape[0], testX.shape[1], testX.shape[2], 1))


In [10]:
print(trainX.shape)
print(trainY.shape)
print(validationX.shape)
print(validationY.shape)

(75160, 128, 64, 1)
(75160, 5)
(9045, 128, 64, 1)
(9045, 5)


## Basic CNN

In [11]:
indim_x = trainX.shape[1]
indim_y = trainX.shape[2]

In [13]:
model = simple_cnn((indim_x,indim_y), conv_layers=[16, 8])

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 62, 16)       160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 31, 16)        0         
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 29, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6720)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               6

In [ ]:
model.fit(trainX, trainY, batch_size = 128, validation_data=(validationX, validationY), epochs=25)

Train on 75160 samples, validate on 9045 samples
Epoch 1/25
75160/75160 [==============================] - 17s 230us/sample - loss: 2.7786 - accuracy: 0.4889 - val_loss: 1.2384 - val_accuracy: 0.5100
Epoch 2/25
75160/75160 [==============================] - 14s 188us/sample - loss: 1.0038 - accuracy: 0.6216 - val_loss: 1.1982 - val_accuracy: 0.5127
Epoch 3/25
75160/75160 [==============================] - 14s 188us/sample - loss: 0.9459 - accuracy: 0.6427 - val_loss: 1.1146 - val_accuracy: 0.5547
Epoch 4/25
75160/75160 [==============================] - 14s 187us/sample - loss: 0.8961 - accuracy: 0.6608 - val_loss: 1.0622 - val_accuracy: 0.5742
Epoch 5/25
75160/75160 [==============================] - 14s 192us/sample - loss: 0.8549 - accuracy: 0.6761 - val_loss: 1.0332 - val_accuracy: 0.5926
Epoch 6/25
75160/75160 [==============================] - 14s 189us/sample - loss: 0.8195 - accuracy: 0.6908 - val_loss: 0.9368 - val_accuracy: 0.6347
Epoch 7/25
75160/75160 [=====================

In [56]:
model.evaluate(testX, testY, verbose=2)

1685/1 - 1s - loss: 0.9238 - accuracy: 0.6617


[0.996642955226785, 0.66172105]